# Prepare DoS Hulk Attack Dataset

## Overview:

This notebook will focus on creating a DoS Hulk attack dataset based on a small sample of data collected by performing real DoS HTTP GET Flood attacks in a controlled environment.<br>
The dataset that this notebook creates closely represents real-world data and was used to train our SVM model.<br>  
It is worth noteing that the sample dataset we collected does not contain any missing values or any outliers due to the fact we tested each part of the collection process and verified that it is correct.<br>
In this notebook we have generated an attack dataset with 9,000 flows of the DoS HTTP GET Flood attack based on the samples we collected when running a DoS HTTP GET Flood attacks in various configurations using the well known DoS Hulk tool when the victim web server was online.<br> 
The victim web server was a regular Flask web server.

## Imports & Global Variables:

In [86]:
import pandas as pd
import numpy as np
import random

NUM_OF_ROWS = 9000
ATTACK_NAME = 'DoS'

In [87]:
# the following command will make it so that when we print the dataframe we will see all the columns
pd.set_option('display.max_columns', None)

---

## Load the sample dataset:

In [88]:
# import the attack sample dataset
dos_samples = pd.read_csv('dos_hulk_samples.csv')
print(f'Dataset Shape: {dos_samples.shape}')
dos_samples

Dataset Shape: (22, 26)


,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,113.448176,54,490,119.397510,14255.765358,792575,456,79.448176,20,119.397510,0,0.0,0,0.0,46.704391,0.0,105.578127,2819,5896,1941,3.921313,2544.045958,0.080743,0.000393,0.001147
1,1,111.399578,54,492,118.554999,14055.287716,770745,458,77.399578,20,118.554999,0,0.0,0,0.0,45.408817,0.0,97.897244,2411,5932,2245,4.324354,2302.771733,0.041172,0.000434,0.001078
2,1,114.086974,54,490,122.233724,14941.083217,799268,456,80.086974,20,122.233724,0,0.0,0,0.0,48.782766,0.0,102.260491,1790,6418,2440,3.992678,2499.575369,0.052491,0.000400,0.000896
3,1,115.067034,54,488,123.774303,15320.077971,809049,454,81.067034,20,123.774303,0,0.0,0,0.0,50.176052,0.0,95.881607,1477,6596,2554,5.106365,1954.423561,0.100327,0.000512,0.001462
4,1,113.853212,54,488,122.117633,14912.716396,798053,454,79.853212,20,122.117633,0,0.0,0,0.0,48.652892,0.0,103.724071,1767,6383,2478,4.209765,2374.004139,0.047036,0.000421,0.000916
5,1,117.572573,54,489,125.957802,15865.367856,834890,455,83.572573,20,125.957802,0,0.0,0,0.0,52.133133,0.0,103.893728,1520,6990,2320,4.390662,2275.283341,0.070261,0.000440,0.001078
6,1,116.129676,54,490,122.376793,14976.079363,818915,456,82.129676,20,122.376793,0,0.0,0,0.0,49.261559,0.0,104.147908,2318,6829,1677,10.304537,967.632019,4.278063,0.001034,0.044491
7,1,110.064299,54,489,116.821153,13647.181842,758285,455,76.064299,20,116.821153,0,0.0,0,0.0,43.861270,0.0,104.432585,2632,5751,2172,3.477777,2866.486457,0.037094,0.000349,0.000705
8,1,106.599037,54,490,111.687568,12474.112822,723885,456,72.599037,20,111.687568,0,0.0,0,0.0,39.632233,0.0,85.092865,3146,5531,1760,9.178418,1086.352771,2.364858,0.000921,0.027002
9,1,104.720705,54,489,109.305360,11947.661642,701903,455,70.720705,20,109.305360,0,0.0,0,0.0,38.039194,0.0,84.070308,3115,5297,1848,16.610516,597.513051,1.989192,0.001674,0.032656


### Find the columns that we need to synthesis data for:

In [ ]:
columns_to_gather = dos_samples.replace(0, np.nan) #replace all 0 values with null
columns_to_gather = columns_to_gather.dropna(how = 'all', axis = 1).columns.tolist() #remove all columns where there are null values
columns_to_gather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Packet Length Std',
 'Packet Length Variance',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Fwd Packet Length Std',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'ACK Flag Count',
 'RST Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

### Find an approximate minimum and maximum values of each column:

In [90]:
# find the minimum and maximum values for each column, scale the range (reduce min by 15% and increase max by 15%), and store the results in a dictionary.
min_max_dict = {col: (dos_samples[col].min() * 0.85, dos_samples[col].max() * 1.15) for col in columns_to_gather}
min_max_dict['Number of Ports'] = (1, 1)

# print the min max dictionary
for col, (min_val, max_val) in min_max_dict.items():
    print(f'{col:<30} | Min: {min_val:.2f} | Max: {max_val:.2f}')

Number of Ports                | Min: 1.00 | Max: 1.00
Average Packet Length          | Min: 84.26 | Max: 135.21
Packet Length Min              | Min: 45.90 | Max: 62.10
Packet Length Max              | Min: 413.95 | Max: 565.80
Packet Length Std              | Min: 85.70 | Max: 144.85
Packet Length Variance         | Min: 8640.52 | Max: 18245.17
Total Length of Fwd Packet     | Min: 446822.05 | Max: 960123.50
Fwd Packet Length Max          | Min: 385.05 | Max: 526.70
Fwd Packet Length Mean         | Min: 55.36 | Max: 96.11
Fwd Packet Length Min          | Min: 17.00 | Max: 23.00
Fwd Packet Length Std          | Min: 85.70 | Max: 144.85
Fwd Segment Size Avg           | Min: 26.95 | Max: 59.95
Subflow Fwd Bytes              | Min: 60.07 | Max: 121.41
SYN Flag Count                 | Min: 1255.45 | Max: 4686.25
ACK Flag Count                 | Min: 3468.85 | Max: 8038.50
RST Flag Count                 | Min: 935.00 | Max: 3061.30
Flow Duration                  | Min: 2.96 | Max: 47.21
Pa

### Create the base attack dataset (full of zeros):

In [ ]:
# creating an empty dataframe before adding values to it
dos_dataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dos_samples.columns))), columns = dos_samples.columns)
dos_dataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Find the columns with constant zero values based on samples:

In [93]:
# adding zeros to all columns that should not have any values
zero_columns = [col for col in dos_samples.columns if col not in columns_to_gather]
for col in zero_columns:
    dos_dataset[col] = int(0)
zero_columns

['Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg']

In [94]:
dos_dataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

## Filling in values based on collected samples:

## Calculate and fill values into columns that have a certain correlation between them:

A correlation between two or more columns is common in our dataset since most features are inherently related. All of them are derived from network packet traffic.<br>
For example, as the **flow duration increases**, the **packets per second** is likely to decrease. This occurs because each flow has an upper limit on duration, after which data collection stops and a new flow begins.<br>  
Similarly, the **Inter-Arrival Time (IAT)** of packets within a flow is influenced by the flow duration. Given these dependencies, <br>
the attack dataset should generate data for these columns collectively, ensuring that their inherent correlations are maintained.

### Correlation between 'SYN Flag Count' and all the following: 'ACK Flag Count', 'RST Flag Count':

In [ ]:
# finding the correlation between the 'SYN Flag Count' column to the rest of the columns in order to create new data
first_correlation = ['SYN Flag Count', 'ACK Flag Count', 'RST Flag Count']
independent_col = dos_samples[first_correlation[0]].values.reshape(-1, 1) #column 'SYN Flag Count'
dependent_cols = dos_samples[first_correlation[1:]].values 

# using least squares regression to find scaling factors that best approximate the relationship between 'SYN Flag Count' and the rest
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('ACK Flag Count', np.float64(1.9518394723650978))
('RST Flag Count', np.float64(0.6629111435788864))


After finding the scaling factors we can apply some randomness when generating values for the attack dataset in order to generate better data (without many duplications).<br>
We add randomness by creating a modified scaling factor, which introduces controlled variations in the generated values.<br>
This is done by selecting a small random delta (between 10% and 20% of the factor) and adding or subtracting it from the original scaling factor.<br>
As a result, the generated data maintains realistic correlations while avoiding exact duplicates.

In [ ]:
dos_dataset['SYN Flag Count'] = np.random.randint(min_max_dict['SYN Flag Count'][0]*0.85, min_max_dict['SYN Flag Count'][1]*1.15, NUM_OF_ROWS)

# generate new data by scaling the original correlated column value using the updated factor
for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all generated scaling factors
        delta = random.uniform(factor * 0.1, factor * 0.2)
        updated_factor = factor + random.choice([-1, 1]) * delta
        dos_dataset.loc[index, col] = int(row['SYN Flag Count'] * updated_factor)

In [97]:
dos_dataset.head(10)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3856,8549.0,2989.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3857,8503.0,2237.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4268,6692.0,3134.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3284,5657.0,1875.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3679,8551.0,2758.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,5155,11496.0,2752.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,5094,8024.0,3918.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1378,3151.0,1083.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1729,4042.0,1028.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1510,3431.0,861.0,0.0,0.0,0.0,0.0,0.0


### Correlation between 'Flow Duration' and all of the following: 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std':

In [ ]:
# generate random values for the 'Flow Duration' column
rand_values = np.random.uniform(min_max_dict['Flow Duration'][0]*0.9, min_max_dict['Flow Duration'][1]*1.05, size = NUM_OF_ROWS)

# assign the random values
dos_dataset['Flow Duration'] = rand_values

In [ ]:
# finding the correlation between the 'Flow Duration' column to the rest of the columns in order to create new data
second_correlation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dos_samples[second_correlation[0]].values.reshape(-1, 1) #column 'Flow Duration'
dependent_cols = dos_samples[second_correlation[1:]].values 

# using least squares regression to find scaling factors that best approximate the relationship between 'Flow Duration' and the rest of the columns in second_correlation
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(second_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packets Per Second', np.float64(29.47374840796544))
('IAT Max', np.float64(0.28959035080285817))
('IAT Mean', np.float64(0.00010689125614908151))
('IAT Std', np.float64(0.0035898854422333083))


In [100]:
# calculate the average correlation between flow duration and packets per second by multiplying their corresponding values from both columns and then calculate the average.
duration_to_packets_corr = [x * y for x, y in zip(dos_samples['Flow Duration'].values, dos_samples['Packets Per Second'].values)]
duration_to_packets_corr = np.mean(duration_to_packets_corr)
duration_to_packets_corr

np.float64(9827.272727272728)

And again here after finding the scaling factors we add some randomness and generate the data

In [ ]:
# calculate a random small delta of the factor for some randomness
for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Flow Duration'
        if col == 'Packets Per Second':
            delta = random.uniform(duration_to_packets_corr * 0.035, duration_to_packets_corr * 0.175)
            updated_factor = duration_to_packets_corr + random.choice([-1, 1]) * delta
            dos_dataset.loc[index, col] = updated_factor / row['Flow Duration']
        else:
            if col == 'IAT Std' or col == 'IAT Max':
                delta = random.uniform(factor * 0.7, factor * 0.95)
                updated_factor = factor + random.choices([-1, 1], weights=[2, 1], k=1)[0] * delta
            else:
                delta = random.uniform(factor * 0.05, factor * 0.15) 
                updated_factor = factor + random.choice([-1, 1]) * delta
            dos_dataset.loc[index, col] = row['Flow Duration'] * updated_factor

### Correlation between 'Average Packet Length' and all of the following:<br>'Packet Length Std', 'Packet Length Variance', 'Total Length of Fwd Packet', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Fwd Segment Size Avg', 'Subflow Fwd Bytes':

In [ ]:
# finding the correlation between the 'Average Packet Length' column to the rest of the columns in order to create new data
third_correlation = ['Average Packet Length', 'Packet Length Std', 'Packet Length Variance', 'Total Length of Fwd Packet', 
                    'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Fwd Segment Size Avg', 'Subflow Fwd Bytes']
independent_col = dos_samples[third_correlation[0]].values.reshape(-1, 1) #column 'Average Packet Length'
dependent_cols = dos_samples[third_correlation[1:]].values 

# using least squares regression to find scaling factors that best approximate the relationship between Average Packet Length' and the rest of the columns in third_correlation
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(third_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packet Length Std', np.float64(1.053481965176044))
('Packet Length Variance', np.float64(122.15926993685173))
('Total Length of Fwd Packet', np.float64(6798.663741787297))
('Fwd Packet Length Mean', np.float64(0.6901644819681063))
('Fwd Packet Length Std', np.float64(1.053481965176044))
('Fwd Segment Size Avg', np.float64(0.39446115648863966))
('Subflow Fwd Bytes', np.float64(0.8079142101649653))


And again here after finding the scaling factors we add some randomness and generate the data

In [ ]:
dos_dataset['Average Packet Length'] = np.random.uniform(min_max_dict['Average Packet Length'][0]*0.85, min_max_dict['Average Packet Length'][1]*1.15, NUM_OF_ROWS)

for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Average Packet Length'
        delta = random.uniform(factor * 0.1, factor * 0.2) 
        updated_factor = factor + random.choice([-1, 1]) * delta
        dos_dataset.loc[index, col] = row['Average Packet Length'] * updated_factor

### Then we insert data into columns that are independant of each other:

In [ ]:
independant_columns = ['Packet Length Min', 'Packet Length Max', 'Fwd Packet Length Max', 'Fwd Packet Length Min']

for col in independant_columns:
    dos_dataset[col] = (np.random.uniform(min_max_dict[col][0]*0.85, min_max_dict[col][1]*1.15, NUM_OF_ROWS)).astype(int)

dos_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,143.999988,43,581,132.614974,14298.267153,8.363211e+05,433,81.817293,19,179.280233,0,0,0,0,65.982485,0,128.964501,3856,8549.0,2989.0,27.273295,394.311434,13.559343,0.003165,0.025422
1,0.0,110.270908,48,381,127.977951,14870.780421,8.272540e+05,362,84.867655,20,94.340815,0,0,0,0,51.514026,0,71.571780,3857,8503.0,2237.0,8.656745,1049.495426,0.630356,0.000995,0.002928
2,0.0,107.921209,65,408,125.256376,14707.764774,6.074138e+05,465,86.389490,20,101.049117,0,0,0,0,49.383228,0,100.120160,4268,6692.0,3134.0,40.425315,221.121306,1.244499,0.004074,0.012448
3,0.0,126.573974,43,368,107.083173,13285.937952,7.721926e+05,454,75.293580,19,117.539721,0,0,0,0,59.732637,0,87.315971,3284,5657.0,1875.0,28.911416,289.739028,16.019787,0.002762,0.195719
4,0.0,112.950871,45,544,100.434954,15203.642189,6.782500e+05,555,86.500839,25,133.332326,0,0,0,0,50.789450,0,103.401877,3679,8551.0,2758.0,10.316516,897.379823,0.792448,0.001227,0.070531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,0.0,100.536289,53,617,120.568705,10902.789862,5.791950e+05,529,57.056460,23,126.641072,0,0,0,0,46.732323,0,93.976013,4860,8275.0,2671.0,19.381756,423.578171,10.403962,0.001928,0.012093
8996,0.0,142.418927,51,395,177.946918,20833.205320,1.065814e+06,586,111.059750,19,122.270454,0,0,0,0,66.816234,0,95.992936,2698,4739.0,2059.0,21.462094,431.768262,0.939324,0.002152,0.019421
8997,0.0,100.115796,68,381,121.310543,13851.689434,5.492241e+05,579,82.702169,23,87.135356,0,0,0,0,45.408128,0,93.621596,2564,4026.0,1475.0,8.013739,1393.975331,4.199310,0.000755,0.005045
8998,0.0,128.235791,40,638,114.550553,13264.842889,9.697406e+05,432,98.310541,16,109.142794,0,0,0,0,43.317770,0,88.567680,4320,7324.0,2414.0,13.401127,846.362669,6.909626,0.001333,0.008593


---

## Adding the Label column:

In [106]:
# adding number of ports and a label to the dataset
dos_dataset['Number of Ports'] = np.full(shape = NUM_OF_ROWS, fill_value = 1, dtype=int)
dos_dataset['Label'] = ATTACK_NAME

## Validate that the generated data looks valid by comparing the samples with the generated dataset:

In [107]:
dos_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std,Label
0,1,143.999988,43,581,132.614974,14298.267153,8.363211e+05,433,81.817293,19,179.280233,0,0,0,0,65.982485,0,128.964501,3856,8549.0,2989.0,27.273295,394.311434,13.559343,0.003165,0.025422,DoS
1,1,110.270908,48,381,127.977951,14870.780421,8.272540e+05,362,84.867655,20,94.340815,0,0,0,0,51.514026,0,71.571780,3857,8503.0,2237.0,8.656745,1049.495426,0.630356,0.000995,0.002928,DoS
2,1,107.921209,65,408,125.256376,14707.764774,6.074138e+05,465,86.389490,20,101.049117,0,0,0,0,49.383228,0,100.120160,4268,6692.0,3134.0,40.425315,221.121306,1.244499,0.004074,0.012448,DoS
3,1,126.573974,43,368,107.083173,13285.937952,7.721926e+05,454,75.293580,19,117.539721,0,0,0,0,59.732637,0,87.315971,3284,5657.0,1875.0,28.911416,289.739028,16.019787,0.002762,0.195719,DoS
4,1,112.950871,45,544,100.434954,15203.642189,6.782500e+05,555,86.500839,25,133.332326,0,0,0,0,50.789450,0,103.401877,3679,8551.0,2758.0,10.316516,897.379823,0.792448,0.001227,0.070531,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,1,100.536289,53,617,120.568705,10902.789862,5.791950e+05,529,57.056460,23,126.641072,0,0,0,0,46.732323,0,93.976013,4860,8275.0,2671.0,19.381756,423.578171,10.403962,0.001928,0.012093,DoS
8996,1,142.418927,51,395,177.946918,20833.205320,1.065814e+06,586,111.059750,19,122.270454,0,0,0,0,66.816234,0,95.992936,2698,4739.0,2059.0,21.462094,431.768262,0.939324,0.002152,0.019421,DoS
8997,1,100.115796,68,381,121.310543,13851.689434,5.492241e+05,579,82.702169,23,87.135356,0,0,0,0,45.408128,0,93.621596,2564,4026.0,1475.0,8.013739,1393.975331,4.199310,0.000755,0.005045,DoS
8998,1,128.235791,40,638,114.550553,13264.842889,9.697406e+05,432,98.310541,16,109.142794,0,0,0,0,43.317770,0,88.567680,4320,7324.0,2414.0,13.401127,846.362669,6.909626,0.001333,0.008593,DoS


In [108]:
dos_samples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,22.0,22.000000,22.0,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.0,22.000000,22.0,22.0,22.0,22.0,22.000000,22.0,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,1.0,109.541236,54.0,489.272727,115.336644,13344.256197,743456.272727,455.272727,75.541236,20.0,115.336644,0.0,0.0,0.0,0.0,43.067228,0.0,88.236516,2693.954545,5715.863636,1978.409091,14.334490,1273.183737,3.416355,0.001497,0.042374
std,0.0,4.723316,0.0,1.031957,6.610693,1503.443014,66624.894863,1.031957,4.723316,0.0,6.610693,0.0,0.0,0.0,0.0,5.371901,0.0,12.216166,687.545218,696.524932,410.466594,11.577552,890.026163,4.604678,0.001294,0.055395
min,1.0,99.131087,54.0,487.000000,100.823173,10165.312267,525673.000000,453.000000,65.131087,20.0,100.823173,0.0,0.0,0.0,0.0,31.706232,0.0,70.674872,1477.000000,4081.000000,1100.000000,3.477777,196.600053,0.037094,0.000349,0.000705
25%,1.0,106.700900,54.0,489.000000,111.827633,12505.478401,710284.500000,455.000000,72.700900,20.0,111.827633,0.0,0.0,0.0,0.0,39.767573,0.0,78.726481,2327.500000,5252.000000,1742.750000,4.569588,465.317607,0.072882,0.000458,0.001226
50%,1.0,109.955870,54.0,489.000000,116.146897,13490.556399,757735.000000,455.000000,75.955870,20.0,116.146897,0.0,0.0,0.0,0.0,43.302417,0.0,86.390381,2894.000000,5704.000000,1904.000000,8.535311,1174.495592,1.109663,0.000856,0.015098
75%,1.0,113.082372,54.0,490.000000,120.085389,14420.658297,788886.750000,456.000000,79.082372,20.0,120.085389,0.0,0.0,0.0,0.0,46.672949,0.0,101.169679,3118.000000,6270.250000,2301.250000,21.099838,2195.068396,5.987090,0.002152,0.080042
max,1.0,117.572573,54.0,492.000000,125.957802,15865.367856,834890.000000,458.000000,83.572573,20.0,125.957802,0.0,0.0,0.0,0.0,52.133133,0.0,105.578127,4075.000000,6990.000000,2662.000000,41.052888,2866.486457,16.314272,0.005087,0.200255


In [109]:
dos_dataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,9000.0,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9.000000e+03,9000.000000,9000.000000,9000.000000,9000.000000,9000.0,9000.0,9000.0,9000.0,9000.000000,9000.0,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000
mean,1.0,113.728216,54.623556,499.454889,119.851217,13886.087033,7.714856e+05,465.551889,78.642888,19.981667,119.698027,0.0,0.0,0.0,0.0,44.837718,0.0,91.834174,3233.058222,6306.811222,2139.810111,26.253608,608.849607,5.410444,0.002805,0.068292
std,0.0,24.305650,9.396269,86.602343,31.911118,3661.666226,2.040588e+05,79.768450,20.773281,3.470959,31.719980,0.0,0.0,0.0,0.0,11.838828,0.0,24.184717,1243.607256,2640.983349,892.261156,13.587910,602.449590,7.231789,0.001487,0.089891
min,1.0,71.627626,39.000000,351.000000,60.661176,7030.358623,3.935407e+05,327.000000,39.995059,14.000000,60.600924,0.0,0.0,0.0,0.0,22.671010,0.0,46.667908,1067.000000,1700.000000,571.000000,2.660855,165.335501,0.039042,0.000246,0.000664
25%,1.0,92.680849,47.000000,425.000000,94.986685,11055.245317,6.122393e+05,396.000000,62.691130,17.000000,95.170298,0.0,0.0,0.0,0.0,35.710468,0.0,73.195471,2161.500000,4085.750000,1397.000000,14.472987,257.865865,0.806296,0.001518,0.010266
50%,1.0,113.830048,55.000000,499.000000,117.668298,13600.624864,7.565051e+05,466.000000,76.960093,20.000000,117.028812,0.0,0.0,0.0,0.0,43.979362,0.0,90.011861,3244.000000,6176.000000,2086.500000,26.267718,373.947874,1.788311,0.002778,0.023193
75%,1.0,134.925076,63.000000,575.000000,140.469178,16221.192398,9.027619e+05,534.000000,92.030914,23.000000,139.944360,0.0,0.0,0.0,0.0,52.381352,0.0,106.982243,4311.000000,8211.000000,2789.000000,38.179275,678.172933,7.212939,0.004010,0.094063
max,1.0,155.488056,71.000000,650.000000,195.922546,22733.774337,1.267428e+06,605.000000,128.524281,26.000000,195.222846,0.0,0.0,0.0,0.0,73.213301,0.0,149.317068,5388.000000,12566.000000,4276.000000,49.562732,4240.403230,27.725649,0.006055,0.346365


---

## At the end we save the dataset as a CSV file

In [110]:
print(f'Attack Dataset Shape: {dos_dataset.shape}')

Attack Dataset Shape: (9000, 27)


In [ ]:
# save the dataset
dos_dataset.to_csv('dos_hulk_dataset.csv', index=False)

---